## Load and clean the data

In [5]:
# dataset is taken from Kaggle and downloaded from here:
# https://www.kaggle.com/datasets/thomasnibb/amsterdam-house-price-prediction
# license is CC0: Public Domain - can copy, modify, distribute and perform the work, 
# even for commercial purposes, all without asking permission.

import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

df = pd.read_csv("HousingPrices-Amsterdam-August-2021.csv")

# STEP 1 - make columns look good:
df.columns = df.columns.str.lower()

used_cols = [ 'address', 'zip',	'price', 'area', 'room']
df = df[used_cols]
df.head()

,address,zip,price,area,room
0,"Blasiusstraat 8 2, Amsterdam",1091 CR,685000.0,64,3
1,"Kromme Leimuidenstraat 13 H, Amsterdam",1059 EL,475000.0,60,3
2,"Zaaiersweg 11 A, Amsterdam",1097 SM,850000.0,109,4
3,"Tenerifestraat 40, Amsterdam",1060 TH,580000.0,128,6
4,"Winterjanpad 21, Amsterdam",1036 KN,720000.0,138,5
